### AWS 관측지점 데이터
**awsp_df**: 관측 지점에 관한 데이터를 가지고 있는 데이터프레임.
* Index: STN(지점번호)
* STN_KO: 지역명
* LAW_ID(법정동코드): 10자리로 된 행정구역 상 코드
* 소재지: 소재지 주소(읍면리 단위)

In [ ]:
import pandas as pd

awsp_df = pd.read_excel("./data/aws_관측지점_위치.xlsx").dropna()
awsp_df

In [ ]:
new_stn = pd.to_numeric(awsp_df["STN(지점번호)"])
awsp_df["STN(지점번호)"] = new_stn
awsp_df.set_index("STN(지점번호)", inplace=True)
awsp_df

In [ ]:
awsp_df.loc[400]

### 서울 침수 데이터

In [ ]:
import pandas as pd

seoul_df = pd.read_csv("./data/seoul_flood_data_2022.csv")
seoul_df

In [ ]:
import geopandas as gpd

seoul_gpdf = gpd.read_file("./data/Seoul_Flood_2022/서울시_2022.shp", encoding="EUC-KR")
seoul_gpdf.head()

In [ ]:
seoul_gpdf["centroid"] = seoul_gpdf.centroid.to_crs(epsg=4326)
seoul_gpdf["lat"] = seoul_gpdf["centroid"].x
seoul_gpdf["lon"] = seoul_gpdf["centroid"].y
seoul_gpdf = seoul_gpdf.to_crs(epsg=4326).drop(columns=['OBJECTID'])
seoul_gpdf.head()

In [ ]:
seoul_gpdf[seoul_gpdf.isna().any(axis=1)]

In [ ]:
import matplotlib.pyplot as plt

base = seoul_gpdf.plot(color="white", edgecolor="k")
base.set_axis_off()
base.set_title("Seoul Flood Map")
plt.show()

In [ ]:
seoul_gpdf = seoul_gpdf[["F_SHIM", "lat", "lon"]]
seoul_gpdf

In [ ]:
import pydeck as pdk

# Define a layer to display on a map
layer = pdk.Layer('ColumnLayer', # GridLayer, , HeatmapLayer
                  seoul_gpdf, get_position='[lon, lat]',
                  extruded=True, get_elevation='[F_SHIM]', elevation_scale= 1,
                  elevation_range=[0, 10],
                  radius= 25, #헥사곤 기둥의 반경 길이
                  coverage=1, #헥사곤 기중의 상대적 크기 1은 꽉 채운 상태
                  opacity= 0.4,
                  pickable=True,
                  auto_highlight=True
)
# Set the viewport location
view_state = pdk.ViewState(
 longitude=126.96,
 latitude=37.60,
 zoom=12,
 min_zoom=5,
 max_zoom=15,
 pitch=25,
 bearing=-15)
# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state, map_style='mapbox://styles/mapbox/dark-v9') 
r.to_html('./result/pop_pydeck.html', notebook_display=True, iframe_width=900, iframe_height=500)
# r.show()

In [ ]:
import plotly.figure_factory as ff
import plotly.express as px
from keys import MAPBOX_API_KEY

px.set_mapbox_access_token(MAPBOX_API_KEY)
df = px.data.carshare()

fig = ff.create_hexbin_mapbox(
    data_frame = seoul_gpdf, lat="lat", lon="lon",
    color="F_SHIM", #유동인구수를 색상 단계에 반영
    nx_hexagon=20, #헥사곤 개수 = 숫자가 커질수록 헥사곤 크기 작아짐
    opacity=0.6, labels={"color":"Point Count"},
    min_count=0.01, #최소 기준
    show_original_data=True,
    original_data_marker=dict(size=2, opacity=0.4, color="black")
)
fig.update_layout(margin=dict(b=0, t=0, l=0, r=0))
fig.show()